### RunnablePasthrough

- **RunnablePassthrough Functionality**:
  - `RunnablePassthrough` in LangChain is utilized to transfer inputs either unchanged or with added extra keys.
  - It's commonly paired with `RunnableParallel` to assign new keys in a map.

- **Key Features of RunnablePassthrough**:
  1. **Standard Use**: When used alone, it simply passes the input through unchanged.
  2. **Assign Functionality**: When used with `assign`, it takes the input and adds extra arguments specified in the assign function.

- **Example Scenario**:
  - In a practical application:
    - Using `RunnablePassthrough()` on its own results in the input being passed through directly.
    - Applying `RunnablePassthrough.assign` with a function modifies the input by adding specified elements.
    - The example demonstrates assigning and modifying keys in a map, such as multiplying a numerical value and adding a new element.


In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
  passed=RunnablePassthrough(),
  extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
  modified=lambda x: x["num"] +1
)

runnable.invoke(
  {
    "num": 1
  }
)

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

### Retrieval example

- **Integration of RunnablePassthrough and RunnableMap**:
  - In the given example, `RunnablePassthrough` is used along with `RunnableMap` to handle a specific input format for a prompt.
  - The prompt expects a map containing "context" and "question" keys, while the actual user input includes only the question.
  - To address this, the `retriever` is used to obtain the context, and `RunnablePassthrough` ensures the user's question is included under the "question" key.
  - This setup facilitates the correct processing of both context and user question in the chain, enabling accurate and context-aware response generation.
  - `RunnablePassthrough` plays a crucial role in this chain by seamlessly forwarding the user's question to the appropriate components.

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

vector_store = FAISS.from_texts(
    [ "Harrison worked at kensho"],
    embedding=OpenAIEmbeddings(),
)
retreiver = vector_store.as_retriever()
model = ChatOpenAI()


template = """
Answer the question based only on the following context: {context}.

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retreiver, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("where did harrison work")

'Harrison worked at Kensho.'